In [1]:
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)


In [3]:
query = "select * from rentals_may"
rentals_may=pd.read_sql_query(query, engine)
rentals_may.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [4]:
query = "select * from rentals_june"
rentals_june=pd.read_sql_query(query, engine)
rentals_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


In [44]:
#no. of rentals in may by customer
may_totals = pd.DataFrame(rentals_may["customer_id"].value_counts())
may_totals


,customer_id
197,8
109,7
506,7
161,6
274,6
...,...
425,1
426,1
427,1
428,1


In [40]:
may_totals["cust_id"] = may_totals.index
may_totals

,customer_id,cust_id
197,8,197
109,7,109
506,7,506
161,6,161
274,6,274
...,...,...
425,1,425
426,1,426
427,1,427
428,1,428


In [15]:
#SQL query for same question

query = "select customer.first_name, customer.last_name, count(rentals_may.customer_id) as no_rentals_may \
from customer inner join rentals_may \
ON customer.customer_id = rentals_may.customer_id \
group by customer.customer_id order by no_rentals_may desc"

pd.read_sql_query(query, engine)

,first_name,last_name,no_rentals_may
0,SUE,PETERS,8
1,LESLIE,SEWARD,7
2,EDNA,WEST,7
3,CASSANDRA,WALTERS,6
4,MINNIE,ROMERO,6
...,...,...,...
502,JOEL,FRANCISCO,1
503,JACK,FOUST,1
504,DOROTHY,TAYLOR,1
505,ANITA,MORALES,1


In [35]:
#no. of rentals in june by customer
june_totals = pd.DataFrame(rentals_june["customer_id"].value_counts())
june_totals

,customer_id
31,11
454,10
329,9
457,9
267,9
...,...
231,1
136,1
436,1
247,1


In [38]:
june_totals["cust_id"] = june_totals.index

In [39]:
june_totals

,customer_id,cust_id
31,11,31
454,10,454
329,9,329
457,9,457
267,9,267
...,...,...
231,1,231
136,1,136
436,1,436
247,1,247


In [18]:
query = "select customer.first_name, customer.last_name, count(rentals_june.customer_id) as no_rentals_june \
from customer inner join rentals_june \
ON customer.customer_id = rentals_june.customer_id \
group by customer.customer_id order by no_rentals_june desc"

pd.read_sql_query(query, engine)

,first_name,last_name,no_rentals_june
0,BRENDA,WRIGHT,11
1,ALEX,GRESHAM,10
2,FRANK,WAGGONER,9
3,DAISY,BATES,9
4,IAN,STILL,9
...,...,...,...
570,FRANCIS,SIKES,1
571,ALLEN,BUTTERFIELD,1
572,GREGORY,MAULDIN,1
573,LAURA,RODRIGUEZ,1


In [42]:
may_june = may_totals.merge(june_totals, how="inner", on="cust_id")     

In [43]:
may_june

,customer_id_x,cust_id,customer_id_y
0,8,197,8
1,7,109,5
2,7,506,5
3,6,161,2
4,6,274,2
...,...,...,...
507,1,425,1
508,1,426,5
509,1,427,5
510,1,428,5


In [105]:
def comparerentals(x):
    
        r=0
        if x == 8:
            r="more rentals in May"
        elif x==3:
            r= "more rentals in June"
        else:
            r= "same in May and June"
        return r

#may_june["which month"] = list(map(comparerentals, may_june))
n=0

while n < len(may_june):
        
        
        if may_june["customer_id_x"][n] > may_june["customer_id_y"][n]:
            may_june["which month"][n] = "More in May than June"
        elif may_june["customer_id_x"][n] < may_june["customer_id_y"][n]:
            may_june["which month"][n] = "More in June than May"
        else:
            may_june["which month"][n] = "Same in May and June"
        n+=1

<ipython-input-105-2e301798009e>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  may_june["which month"][n] = "Same in May and June"
<ipython-input-105-2e301798009e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  may_june["which month"][n] = "More in May than June"
<ipython-input-105-2e301798009e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  may_june["which month"][n] = "More in June than May"


In [106]:
may_june

,customer_id_x,cust_id,customer_id_y,which month
0,8,197,8,Same in May and June
1,7,109,5,More in May than June
2,7,506,5,More in May than June
3,6,161,2,More in May than June
4,6,274,2,More in May than June
...,...,...,...,...
507,1,425,1,Same in May and June
508,1,426,5,More in June than May
509,1,427,5,More in June than May
510,1,428,5,More in June than May


In [107]:
may_june["which month"].value_counts()

More in June than May    358
More in May than June     92
Same in May and June      62
Name: which month, dtype: int64